# Sentiment-Adjusted Topic Series (SentiWS)

In this notebook, I construct sentiment-adjusted topic series. For sentiment extraction, I use the SentiWS dictionary by [Remus, R., Quasthoff, U., and Heyer, G.](https://www.researchgate.net/publication/220746590_SentiWS_-_A_Publicly_Available_German-language_Resource_for_Sentiment_Analysis). 

The process involves identifying the 10 articles with the highest proportion of each topic for each day. I then calculate the average sentiment measure for these articles, which serves as the sentiment value for the topic on that day. Finally, I adjust the daily topic values by multiplying them with the corresponding sentiment measure.

To begin, I load the datasets from Handelsblatt, SZ, Welt, and dpa.

In [1]:
import os
import pandas as pd
from ast import literal_eval

# Set the path variable to point to the 'newspaper_data_processing' directory.
path = os.getcwd().replace('\\nowcasting_with_text\\sentiment', '\\newspaper_data_processing')

# Load pre-processed 'dpa' dataset from a CSV file.
dpa = pd.read_csv(path + '\\dpa\\' + 'dpa_prepro_final.csv', encoding = 'utf-8', sep=';', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

# Keep only the article texts and their respective publication dates.
dpa = dpa[['texts', 'day', 'month', 'year']]

# Load pre-processed 'SZ' dataset from a CSV file.
sz = pd.read_csv(path + '\\SZ\\' + 'sz_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'newspaper_2': 'str',
                                                                                                 'quelle_texts': 'str',
                                                                                                 'page': 'str',
                                                                                                 'rubrics': 'str'})
sz.page = sz.page.fillna('')
sz.newspaper = sz.newspaper.fillna('')
sz.newspaper_2 = sz.newspaper_2.fillna('')
sz.rubrics = sz.rubrics.fillna('')
sz.quelle_texts = sz.quelle_texts.fillna('')

# Keep only the article texts and their respective publication dates.
sz = sz[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Handelsblatt' dataset from a CSV file.
hb = pd.read_csv(path + '\\Handelsblatt\\' + 'hb_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'kicker': 'str',
                                                                                                 'page': 'str',
                                                                                                 'series_title': 'str',
                                                                                                 'rubrics': 'str'})
hb.page = hb.page.fillna('')
hb.series_title = hb.series_title.fillna('')
hb.kicker = hb.kicker.fillna('')
hb.rubrics = hb.rubrics.fillna('')

# Keep only the article texts and their respective publication dates.
hb = hb[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Welt' dataset from a CSV file.
welt = pd.read_csv(path + '\\Welt\\' + 'welt_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'rubrics': 'str',
                                                                                                 'title': 'str'})
welt.title = welt.title.fillna('')
welt.rubrics = welt.rubrics.fillna('')

# Keep only the article texts and their respective publication dates.
welt = welt[['texts', 'day', 'month', 'year']]

# Concatenate the 'dpa', 'sz', 'hb', and 'welt' DataFrames into a single DataFrame 'data'.
data = pd.concat([dpa, sz, hb, welt])

# The number of articles in the final dataset.
print(len(data))

# Sort the data in chronological order.
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
# Reset the index of the DataFrame
data.reset_index(inplace=True, drop=True)
data.head()

3336299


,texts,day,month,year
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,1,1,1991
1,Welajati: Iran bleibt bei einem Krieg am Golf ...,1,1,1991
2,Bush will offenbar seinen Außenminister erneut...,1,1,1991
3,Sperrfrist 1. Januar 1000 HBV fordert umfassen...,1,1,1991
4,Schamir weist Nahost-Äußerungen des neuen EG-P...,1,1,1991


Next, I calculate the sentiment measure for each article as the average sentiment score per word.

To do this, I first load the SentiWs files into a single dictionary mapping every base word and every inflected form to the sentiment score.

In [2]:
def load_sentiws(pos_path, neg_path, encoding="utf-8-sig"):
    """
    Read both positive and negative SentiWS files and
    return a dict {word_or_form_lower: score}, where
    any word appearing in both files gets the average of the two scores.
    """
    lex = {}
    
    # 1) Load all positive words
    with open(pos_path, encoding=encoding) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Split on tabs: ["Abmachung|NN", "0.0040", "Abmachungen"]
            parts = line.split("\t")
            base_pos = parts[0]
            score = float(parts[1].replace(",", "."))
            base = base_pos.split("|")[0].lower()
            # Base form
            lex[base] = score
            # Each inflected form
            if len(parts) >= 3 and parts[2].strip():
                for form in parts[2].split(","):
                    lex[form.lower()] = score
                    
    # 2) Load all negative words
    with open(neg_path, encoding=encoding) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Split on tabs: ["Abmachung|NN", "0.0040", "Abmachungen"]
            parts = line.split("\t")
            base_neg = parts[0]
            score = float(parts[1].replace(",", "."))
            base = base_neg.split("|")[0].lower()
            
            # Helper to merge into lex
            def merge(word):
                w = word.lower()
                if w in lex:
                    lex[w] = (lex[w] + score) / 2.0
                else:
                    lex[w] = score

            merge(base)
            
            # Each inflected form
            if len(parts) >= 3 and parts[2].strip():
                for form in parts[2].split(","):
                    merge(form)       
            
    return lex

In [3]:
import numpy as np

# Load lexicon
lex = load_sentiws("SentiWS_v2.0_Positive.txt",
                   "SentiWS_v2.0_Negative.txt")
vocab = list(lex.keys())
scores = np.array([lex[w] for w in vocab], dtype=float)

Then I calculate the sum of sentiment scores for each article.

In [4]:
import multiprocessing as mp
import score_chunk
from datetime import datetime

# Number of cores to use
NUM_CORE = mp.cpu_count() - 4 

# Split data into chunks for parallel processing
chunk_size = len(data.texts) // NUM_CORE + 1 
text_chunks = [data.texts[i:i + chunk_size] for i in range(0, len(data.texts), chunk_size)]

startTime = datetime.now()

if __name__ == "__main__":
    
    pool = mp.Pool(NUM_CORE)

    # Process each chunk in parallel
    scr_sum_results = pool.starmap(score_chunk.score_chunk, [(chunk, vocab, scores) for chunk in text_chunks])

    # Close and join the pool
    pool.close()
    pool.join()

    # Combine results from all chunks
    scr_sum = np.concatenate(scr_sum_results)

print(datetime.now() - startTime)   

0:02:26.945552


Next, I calculate the total number of words in each article.

In [5]:
import sys
sys.path.append("../uncertainty")

startTime = datetime.now() 

# Import the function calculating the number of words in a text
import count_words_mp

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:38.211132


Finally, I calculate the average sentiment score per word.

In [6]:
# Compute average sentiment per word
# avoid division-by-zero
avg_sent = np.zeros_like(scr_sum, dtype=float)
count_results = np.array(count_results)
mask = count_results > 0
avg_sent[mask] = scr_sum[mask] / count_results[mask]

# Attach sentiment to the DataFrame
data["sentiment"] = avg_sent

In [7]:
data.head()

,texts,day,month,year,sentiment
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,1,1,1991,-0.009476
1,Welajati: Iran bleibt bei einem Krieg am Golf ...,1,1,1991,-0.015265
2,Bush will offenbar seinen Außenminister erneut...,1,1,1991,0.000781
3,Sperrfrist 1. Januar 1000 HBV fordert umfassen...,1,1,1991,-0.004944
4,Schamir weist Nahost-Äußerungen des neuen EG-P...,1,1,1991,0.002576


Now, I incorporate the topic distributions for each article, which were previously computed using the Latent Dirichlet Allocation (LDA) algorithm in the notebook titled `Topic Model Estimation.ipynb`.

In [8]:
# Set the path variable to point to the 'topics' directory.
path = os.getcwd().replace('\\sentiment', '\\topics')

# Load the article topics from a CSV file
article_topics = pd.read_csv(path + '\\article_topic.csv', encoding='utf-8', index_col=0)

# Merge the `data` DataFrame with the `article_topics` DataFrame
data = pd.concat([data, article_topics], axis=1)

I define a function, `get_average_sentiment`, which calculates the average sentiment measure for each topic on a given day. This function selects a specified number of articles with the highest proportions for each topic and computes the average sentiment across these articles. The `calculate_average_sentiment` function applies this calculation to the entire dataset by grouping the data by date and processing each group in parallel using multiprocessing. The results are then combined into a single pandas DataFrame, with the topics as columns and the dates as the index.

In [9]:
from get_average_sentiment import get_average_sentiment

# Convert 'year', 'month', 'day' to datetime
data['date'] = pd.to_datetime(data[['year', 'month', 'day']])

def calculate_average_sentiment(data, n_articles):
    """
    Function to calculate average sentiment for topics based on a specified number of articles.
    """
    # Group data by 'date' to ensure each day stays together
    grouped_by_date = [group for _, group in data.groupby('date')]
    
    # Prepare arguments for starmap (pair each group with the value of n_articles)
    args = [(group, n_articles) for group in grouped_by_date]
    
    if __name__ == "__main__":
        # Create a multiprocessing pool
        pool = mp.Pool(NUM_CORE)

        # Process each group (one day of data) in parallel
        results = pool.starmap(get_average_sentiment, args)

        # Concatenate the results into a single DataFrame
        daily_average_sentiment = pd.concat(results)

        # Close and join the pool
        pool.close()
        pool.join()

    return daily_average_sentiment

In [10]:
startTime = datetime.now() 

# Generate average sentiment for 10 articles
daily_average_sentiment_10 = calculate_average_sentiment(data, n_articles=10)

print(datetime.now()-startTime)

0:03:47.102173


Now I am going to load the daily topics and adjust them using the DataFrame `daily_average_sentiment_10`. The result is a dataframe where topic distributions are multiplied with the average sentiment of each topic for a given day.

In [11]:
# Load the daily topics from a CSV file
daily_topics = pd.read_csv(path + '\\daily_topics.csv', encoding='utf-8')

# Convert year, month, and day into a single date column
daily_topics['date'] = pd.to_datetime(daily_topics[['year','month','day']])
daily_topics.drop(columns=['year', 'month', 'day'], inplace=True)

# Now, set 'date' as index
daily_topics.set_index('date', inplace=True)

In [12]:
# Apply sentiment adjustment to the daily topics
sentiment_adjusted_daily_topics = daily_topics.multiply(daily_average_sentiment_10)

I iterate over each topic and generate a graph comparing the original and sentiment-adjusted values.

In [13]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import os

# Create a directory to save the plots
os.makedirs('topics_plots_SentiWS', exist_ok=True)

# Define the shaded areas for recessions
recessions = [
    ("1992-01-01", "1993-12-31"),  # Post-reunification recession
    ("2001-01-01", "2001-12-31"),  # Dot-com recession
    ("2008-01-01", "2009-12-31"),  # Great Recession
    ("2011-01-01", "2013-12-31")   # European sovereign debt crisis
]

# Calculate the 180-day rolling mean for each series
daily_topics_rm = daily_topics.rolling(window=180).mean()
sentiment_adjusted_daily_topics_10_rm = sentiment_adjusted_daily_topics.rolling(window=180).mean()

# Iterate over each topic
for i in range(daily_topics.shape[1]):
    # Generate the plot
    fig, ax1 = plt.subplots(figsize=(12, 6))
    
    # Plot original topics on the primary y-axis
    ax1.plot(daily_topics_rm.index, daily_topics_rm.iloc[:, i], label='Original Topic', color='black')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Original Topic Proportion', color='black')
    ax1.tick_params(axis='y', labelcolor='black')
    
    # Add shaded areas for recessions
    for start, end in recessions:
        ax1.axvspan(pd.to_datetime(start), pd.to_datetime(end), color='grey', alpha=0.3)
    
    # Create a secondary y-axis for the sentiment-adjusted topics
    ax2 = ax1.twinx()
    ax2.plot(sentiment_adjusted_daily_topics_10_rm.index, sentiment_adjusted_daily_topics_10_rm.iloc[:, i], label='Sentiment-Adjusted Topic', linestyle='--')
    ax2.set_ylabel('Sentiment-Adjusted Topic')
    ax2.tick_params(axis='y')
    
    # Add title and legends
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')
    
    # Format the x-axis to show every year
    ax1.xaxis.set_major_locator(mdates.YearLocator())
    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax1.tick_params(axis='x', rotation=45)
    
    # Save the plot in the 'topics_plots' directory
    plt.savefig('topics_plots_SentiWS/Topic_' + str(i) + '.png')
    
    # Clear the current figure to free memory
    plt.clf()
    
    # Close the current figure to free memory
    plt.close(fig)

In [14]:
# Reset the index
sentiment_adjusted_daily_topics = sentiment_adjusted_daily_topics.reset_index()

# Create 'year', 'month', and 'day' columns
sentiment_adjusted_daily_topics['year'] = sentiment_adjusted_daily_topics['date'].dt.year
sentiment_adjusted_daily_topics['month'] = sentiment_adjusted_daily_topics['date'].dt.month
sentiment_adjusted_daily_topics['day'] = sentiment_adjusted_daily_topics['date'].dt.day

# Drop the old 'index' column which holds the date
sentiment_adjusted_daily_topics = sentiment_adjusted_daily_topics.drop(columns=['date'])

# Reorder the columns to have 'year', 'month', 'day' as the first three columns
cols = ['year', 'month', 'day'] + [col for col in sentiment_adjusted_daily_topics if col not in ['year', 'month', 'day']]
sentiment_adjusted_daily_topics_format = sentiment_adjusted_daily_topics[cols]

# Save sentiment-adjusted topics to a CSV file
sentiment_adjusted_daily_topics_format.to_csv('sentiment_adjusted_daily_topics_SentiWS.csv', encoding='utf-8', index=True)